In [1]:
import keras,os
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D,Dropout, BatchNormalization
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from keras import optimizers
from keras.layers.core import Lambda
from keras import backend as L
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# train_path = 'char_output_extended/train'
# test_path = 'char_output_extended/test'

# train_gendata = ImageDataGenerator(rescale = 1./255,validation_split = 0.15)

# valid_gendata = ImageDataGenerator(rescale = 1./255,validation_split = 0.15)

# test_gendata  = ImageDataGenerator(rescale = 1./255)
# training_dataset  = train_gendata.flow_from_directory(directory = train_path,target_size = (128,128),
#                                                    class_mode = 'categorical',subset = 'training',batch_size = 32)
# validation_dataset = valid_gendata.flow_from_directory(directory = train_path,target_size = (128,128),
#                                                   class_mode = 'categorical',subset = 'validation',batch_size = 32)
# testing_dataset = test_gendata.flow_from_directory(directory = test_path,target_size = (128,128),
#                                                 class_mode = 'categorical',batch_size = 32)


In [9]:
train_path = 'char_output_extended/train'
test_path = 'char_output_extended/test'

train_gendata = ImageDataGenerator(rescale=1./255, validation_split=0.15)
valid_gendata = ImageDataGenerator(rescale=1./255, validation_split=0.15)
test_gendata = ImageDataGenerator(rescale=1./255)

training_dataset = train_gendata.flow_from_directory(directory=train_path, target_size=(128,128),
                                                    class_mode='sparse', subset='training', batch_size=32)
validation_dataset = valid_gendata.flow_from_directory(directory=train_path, target_size=(128,128),
                                                       class_mode='sparse', subset='validation', batch_size=32)
testing_dataset = test_gendata.flow_from_directory(directory=test_path, target_size=(128,128),
                                                  class_mode='sparse', batch_size=32)

Found 2732 images belonging to 48 classes.
Found 470 images belonging to 48 classes.
Found 857 images belonging to 48 classes.


In [10]:
# # Define the model architecture
# model = Sequential()

# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)))
# model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Flatten())

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(48, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# palm_model = model.fit(training_dataset, validation_data=validation_dataset, epochs=40, steps_per_epoch=5, verbose=1)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(120, activation='relu'))

model.add(Dense(84, activation='relu'))

model.add(Dense(48, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
palm_model = model.fit(training_dataset, validation_data=validation_dataset, epochs=40, steps_per_epoch=5, verbose=1)


Epoch 1/40
5/5 [==============================] - 1s 180ms/step - loss: 5.0817 - accuracy: 0.0357 - val_loss: 4.0833 - val_accuracy: 0.0596
Epoch 2/40
5/5 [==============================] - 1s 162ms/step - loss: 4.0595 - accuracy: 0.0562 - val_loss: 3.8184 - val_accuracy: 0.1277
Epoch 3/40
5/5 [==============================] - 1s 161ms/step - loss: 3.8203 - accuracy: 0.0812 - val_loss: 3.7940 - val_accuracy: 0.1128
Epoch 4/40
5/5 [==============================] - 1s 158ms/step - loss: 3.7249 - accuracy: 0.1937 - val_loss: 3.7275 - val_accuracy: 0.1532
Epoch 5/40
5/5 [==============================] - 1s 164ms/step - loss: 3.6946 - accuracy: 0.1250 - val_loss: 3.6635 - val_accuracy: 0.1021
Epoch 6/40
5/5 [==============================] - 1s 160ms/step - loss: 3.6936 - accuracy: 0.1187 - val_loss: 3.5756 - val_accuracy: 0.1766
Epoch 7/40
5/5 [==============================] - 1s 151ms/step - loss: 3.5829 - accuracy: 0.1000 - val_loss: 3.4829 - val_accuracy: 0.1064
Epoch 8/40
5/5 [====

In [6]:
plt.plot(palm_model.history['loss'])
plt.plot(palm_model.history['val_loss'])
plt.title('Checking Fit')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [7]:
import numpy as np
from tensorflow.keras.preprocessing import image


# Define the class labels
class_labels = training_dataset.class_indices
class_labels = {v: k for k, v in class_labels.items()}

# Function to predict class labels for new images
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalize the image

    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)

    return class_labels[predicted_class]

# Example usage
image_path = 'forsplit/folder_1/MaI12_Page4.jpg_char_91.jpg'  # Replace 'path_to_image.jpg' with the actual path to your test image
predicted_label = predict_image(image_path)
print('Predicted class label:', predicted_label)


1/1 [==============================] - 0s 119ms/step
Predicted class label: 3333


In [7]:
# import numpy as np
# from tensorflow.keras.preprocessing import image


# # Load a new image to predict its class
# img_path = 'balanced_dataset/test2.jpg.png'
# img = image.load_img(img_path, target_size=(128, 128))
# img_array = image.img_to_array(img)
# img_array = np.expand_dims(img_array, axis=0)
# img_array /= 255.0

# # Make a prediction
# predictions = model.predict(img_array)
# predicted_class = np.argmax(predictions[0])

# print(predicted_class)

# class_indices = training_dataset.class_indices
# inverse_map = {v: k for k, v in class_indices.items()}
# predicted_label = inverse_map[predicted_class]

# print(predicted_label)

In [8]:
model.save('mytrailmodel.h5')